# Imports

In [2]:
import caffe
import caffe.draw
from caffe.proto import caffe_pb2
from google.protobuf import text_format
import subprocess

# Print net

In [3]:
with open("multiclass_hdf5.prototxt", 'r') as f: print f.read()

name: "multiclass_hdf5"

layer {
  name: "data_hdf5"
  type: "HDF5Data"
  include {
    phase: TRAIN
  }
  hdf5_data_param {
    source: "multiclass_caffe.hdf5.txt"
    batch_size: 10
  }
  top: "data"
  top: "label"
}

layer {
  name: "data_hdf5"
  type: "HDF5Data"
  include {
    phase: TEST
  }
  hdf5_data_param {
    source: "multiclass_caffe_test.hdf5.txt"
    batch_size: 10
  }
  top: "data"
  top: "label"
}

layer {
  name: "hidden_layer01"
  type: "InnerProduct"
  inner_product_param {
    num_output: 20
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
  bottom: "data"
  top: "hl01"
}

layer {
  name: "relu01"
  type: "ReLU"
  bottom: "hl01"
  top: "r01"
}

layer {
  name: "hidden_layer02"
  type: "InnerProduct"
  inner_product_param {
    num_output: 6
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
  bottom: "r01"
  top: "hl02"
}

layer {
  name: "accuracy"
  type: "Accuracy"
  

# Draw net

In [4]:
net = caffe_pb2.NetParameter()
text_format.Merge(open("multiclass_hdf5.prototxt").read(), net)

caffe.draw.draw_net_to_file(net, "multiclass_hdf5.png", "TB")

![](multiclass_hdf5.png)

# Train net

In [17]:
# could not train using pycaffe...

proc = subprocess.Popen(
    ["/data/soft/caffe/build/tools/caffe", "train", "--solver=multiclass_hdf5_solver.prototxt"],
    stderr=subprocess.PIPE)

out = proc.communicate()[1]

print out

/data/soft/caffe/build/tools/caffe: /data/soft/anaconda2/lib/libtiff.so.5: no version information available (required by /usr/lib/x86_64-linux-gnu/libopencv_highgui.so.2.4)
/data/soft/caffe/build/tools/caffe: /data/soft/anaconda2/lib/liblzma.so.5: no version information available (required by /usr/lib/x86_64-linux-gnu/libunwind.so.8)
/data/soft/caffe/build/tools/caffe: /data/soft/anaconda2/lib/liblzma.so.5: no version information available (required by /usr/lib/x86_64-linux-gnu/libavcodec-ffmpeg.so.56)
I0419 11:17:58.279649 15680 caffe.cpp:178] Use CPU.
I0419 11:17:58.279850 15680 solver.cpp:48] Initializing solver from parameters: 
test_iter: 100
test_interval: 500
base_lr: 0.01
display: 100
max_iter: 4000
lr_policy: "fixed"
momentum: 0.9
weight_decay: 0.004
snapshot: 4000
snapshot_prefix: "multiclass_hdf5"
solver_mode: CPU
net: "multiclass_hdf5.prototxt"
I0419 11:17:58.279952 15680 solver.cpp:91] Creating training net from net file: multiclass_hdf5.prototxt
I0419 11:17:58.280109 1568

# Test model

In [18]:
net = caffe.Net('multiclass_hdf5.prototxt',
                'multiclass_hdf5_iter_4000.caffemodel',
                caffe.TEST)

out = net.forward()

print 'Accuracy: ', out['accuracy']

Accuracy:  0.899999976158
